In [1]:
from aiohttp import web
import numpy as np
import json as json
import timeit
from tensorflow.keras.models import model_from_json

In [2]:
class Model():

    initialized = False

    def __init__(self):
        if (Model.initialized == False): Model.init()

    @classmethod
    def init(cls):
        print('init')
        cls.model = cls.load_keras_model("/model/kompetens-model", "1.1")
        cls.initialized = True

    @classmethod
    def load_keras_model(cls, name, model_version = None):
        if model_version: name = name + '-' + model_version
        json_file = open(name + '.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        loaded_model.load_weights(name + '-weights.h5')
        return loaded_model

    def json_to_numpy(self, json_str, key='data'):
        return np.array(json.loads(json_str)[key])

    def numpy_to_json(self, nd_array, key='data'):
        try:
            if nd_array.shape[1] == 1:
                nd_array.reshape(nd_array.shape[0])
            nd_array_list =  { key : nd_array.tolist() }
            return json.dumps(nd_array_list)
        except Exception as e:
            nd_array_list =  { key : [] }
        return json.dumps(nd_array_list)

    def predict(self, indata):
        return Model.model.predict(indata)

    def predict_json(self, json_indata):
        numpy_indata = self.json_to_numpy(json_indata)
        model_prediction = self.predict(numpy_indata)
        return self.numpy_to_json(model_prediction)

In [3]:
async def predict(request):
    model = Model()
    start_time = timeit.default_timer()
    json_indata = await request.json()
    model_prediction = model.predict_json(json_indata)
    print(timeit.default_timer() - start_time)
    return web.json_response(model_prediction)


In [4]:
app = web.Application(client_max_size=10000000)

In [5]:
app.router.add_routes([web.post('/predict', predict)])

In [ ]:
web.run_app(app)

======== Running on http://0.0.0.0:8080 ========
(Press CTRL+C to quit)
init
1.6072732000029646
1.7567156000004616
1.618117699996219
1.4719272999936948
1.5727919999917503
1.6865800999948988
1.8347643000015523
1.4930165000114357
1.4524811000010232
1.5412168000038946
1.7188888999953633
